<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین چهارم (طبقه‌بندی و خوشه‌بندی)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        دسترسی به داده‌ها و مدل‌های ذخیره شده
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        به دلیل حجم بالای فایل‌های ذخیره شده، از قرار دادن آن‌ها در پوشه نوتبوک صرف نظر کرده و با اجرای قطعه کد زیر، از طریق گوگل درایو به آن‌ها دسترسی پیدا می‌کنیم.
    </p>
</div>

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive')
address = "University/Term 6/MIR/Homeworks/HW3"
sys.path.append(f"/content/drive/My Drive/{address}")
%cd /content/drive/My\ Drive/$address

Mounted at /content/drive
/content/drive/My Drive/University/Term 6/MIR/Homeworks/HW3


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [2]:
!pip install bs4
!pip install tqdm
# !pip install pandas
# !pip install requests
!pip install hazm
!pip install unidecode
!pip install datasets
# !pip install fasttext
!pip install scikit-learn
# !pip install pandas
# !pip install nltk
# !pip install torch
!pip install transformers
# !pip install sentence_transformers
# !pip install faiss-cpu
# !pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 4.1 MB/s 
     |████████████████████████████████| 1.4 MB 33.0 MB/s 
     |████████████████████████████████| 233 kB 40.5 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=3cb762fba619b63f6df7b745982465a66554edc64188d852aab740bf9ccaf68a
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154663 sha256=fefb10ccbeea5f07b2f09981746a8133b2b993fceb95d7e62eae4278139b4016
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Su

In [3]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import pickle
import zipfile
import requests
# import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from datasets import Dataset
from tabulate import tabulate
from bs4 import BeautifulSoup
from string import punctuation
from sklearn.cluster import KMeans
from IPython.display import display
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٦٨ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [26]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

CATEGORIES_CLASSES = {
    'Politics': 0,
    'World': 1,
    'Economy': 2,
    'Society': 3,
    'City': 4,
    'Sport': 5,
    'Science': 6,
    'Culture': 7,
    'IT': 8,
    'LifeSkills': 9,
}

CLASSES_CATEGORIES = {v: k for k, v in CATEGORIES_CLASSES.items()}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [8]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    os.remove('dataset.csv')
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [6]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [5]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [23]:
def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


def data_to_text(data):
    return ' '.join([data['title']]).lower()


def get_preprocessed_texts(dataset, preprocessor, mode, save=False):
    preprocessed_texts = []
    if mode == 'process':
        texts = [data_to_text(data) for _, data in dataset.iterrows()]
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    if mode == 'load':
        preprocessed_texts = load_preprocessed_texts()
    if save:
        save_preprocessed_texts(preprocessed_texts)
    return preprocessed_texts

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم، یا داده‌های پیش‌پردازش‌شده‌ی ذخیره‌شده را از فایل مربوطه بازیابی می‌کنیم.
    </p>
</div>

In [24]:
preprocessor = Preprocessor(stopwords_path='stopwords.txt')

In [ ]:
preprocessed_texts = get_preprocessed_texts(dataset, preprocessor, mode='load', save=False)

In [9]:
preprocessed_titles = [' '.join(preprocessor.preprocess(text)) for text in tqdm(dataset.title)]

100%|██████████| 68362/68362 [00:18<00:00, 3720.33it/s]


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با توجه به این که حجم مجموعه داده‌ها باعث ایجاد محدودیت حافظه و زمان برای بعضی از مدل‌ها می‌شود، ناچار هستیم کسری از مجموعه‌ی داده‌ها را به عنوان ورودی برای یادگیری به آن‌ها بدهیم. با اجرای قطعه کد زیر، یک مجموعه داده‌ی کوچک‌تر از روی مجموعه داده‌ی اصلی ایجاد می‌کنیم.
    </p>
</div>

In [ ]:
def get_mini_dataset(len_each_category=400):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts


mini_dataset, mini_preprocessed_texts = get_mini_dataset(1000)

100%|██████████| 9437/9437 [00:01<00:00, 5579.54it/s]


In [ ]:
mini_dataset['category_code'] = mini_dataset['category'].apply(lambda x: CATEGORIES_CLASSES[x])
labels = mini_dataset['category_code'].to_numpy()

In [ ]:
vocabulary = {}

with open('vocabulary.pickle', "rb") as fp:
            vocabulary = pickle.load(fp)


In [ ]:
class TF_IDF:

    def __init__(self, vocabulary):
        self.vectorizer = TfidfVectorizer(vocabulary=vocabulary)
        self.vectors = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        return np.array(dense_vectors), self.words

    def save_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "wb") as file:
            pickle.dump(self.vectors.todense().tolist(), file)

    def load_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "rb") as file:
            return pickle.load(file)

In [ ]:
TF_IDF_model = TF_IDF(vocabulary)
vectors, features = TF_IDF_model.fit_transform_vectorizer(mini_preprocessed_texts)

In [ ]:
train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors, labels,  test_size=0.15, shuffle=True)

In [ ]:
clf = LogisticRegression(max_iter=1000, random_state=0)
clf.fit(train_vectors, train_labels)

LogisticRegression(max_iter=1000, random_state=0)

In [ ]:
headers = ["نام دسته"] + list(CATEGORIES.values())

In [30]:
inv_map = {v: k for k, v in CATEGORIES_CLASSES.items()}

In [ ]:
mean_acc = clf.score(test_vectors, test_labels)
y = clf.predict(test_vectors)
f1_macro = f1_score(test_labels, y, labels=np.arange(10), average='macro')
conf_matrix = confusion_matrix(test_labels, y, labels=np.arange(10))
print(f"Mean Accurracy: {round(mean_acc, 3)}")
print(f"F1_Macro: {round(f1_macro, 3)}")
c_m_df = pd.DataFrame(conf_matrix).rename(columns=inv_map, index=inv_map)
c_m_df

Mean Accurracy: 0.732
F1_Macro: 0.72


,Politics,World,Economy,Society,City,Sport,Science,Culture,IT,LifeSkills
Politics,91,18,3,4,7,2,4,7,0,2
World,14,105,1,8,1,8,12,10,2,2
Economy,12,6,118,15,7,0,2,1,2,0
Society,4,1,5,100,15,0,10,16,1,5
City,6,1,7,6,112,1,5,9,0,3
Sport,4,0,0,4,1,147,0,1,0,1
Science,0,4,0,4,0,0,95,9,4,18
Culture,5,2,1,2,1,1,6,116,0,3
IT,3,6,4,2,2,1,14,5,33,1
LifeSkills,0,2,0,3,0,2,10,8,1,119


In [10]:
dataset['category_code'] = dataset['category'].apply(lambda x: CATEGORIES_CLASSES[x])
labels = dataset['category_code'].to_numpy()

In [6]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-zwnj-base")

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [11]:
new_dataset = pd.DataFrame()
new_dataset['text'] = dataset['title']
new_dataset['label'] = dataset['category_code']

In [12]:
train_data, test_data, train_labels, test_labels = train_test_split(new_dataset, labels,  test_size=0.3, shuffle=True)

In [13]:
val_data, test_data, val_labels, test_labels = train_test_split(test_data, test_labels,  test_size=0.5, shuffle=True)

In [15]:
tokenized_train_data = Dataset.from_pandas(train_data).map(lambda x: tokenizer(x['text'], truncation=True), batched=True)

Parameter 'function'=<function <lambda> at 0x7f7dbc55f170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/48 [00:00<?, ?ba/s]

In [16]:
tokenized_val_data = Dataset.from_pandas(val_data).map(lambda x: tokenizer(x['text'], truncation=True), batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

In [17]:
tokenized_test_data = Dataset.from_pandas(test_data).map(lambda x: tokenizer(x['text'], truncation=True), batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
model = AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-zwnj-base", num_labels=10, id2label=CLASSES_CATEGORIES)

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/b

In [21]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [23]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 47853
  Num Epochs = 5
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 1870


Step,Training Loss
500,0.650200
1000,0.396300
1500,0.299900


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_toke

TrainOutput(global_step=1870, training_loss=0.4074167751373454, metrics={'train_runtime': 1366.321, 'train_samples_per_second': 175.116, 'train_steps_per_second': 1.369, 'total_flos': 4426475904291672.0, 'train_loss': 0.4074167751373454, 'epoch': 5.0})

In [24]:
model.save_pretrained("./new_titles_transformer_model")

Configuration saved in ./new_titles_transformer_model/config.json
Model weights saved in ./new_titles_transformer_model/pytorch_model.bin


In [16]:
model = AutoModelForSequenceClassification.from_pretrained("./new_titles_transformer_model")

In [17]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
from transformers import TextClassificationPipeline

In [19]:
device = torch.cuda.current_device() if torch.cuda.is_available() else -1

In [20]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False, device=device)

In [41]:
QUERIES = [
    "نتیجه توافق برجام",
    "حمله ارتش روسیه به اوکراین",
    "افزایش نرخ تورم در کشور",
    "آمار فوتی‌های کرونا",
    "اقدامات شهرداری تهران",
    "صعود ایران به جام جهانی",
    "بیماری آبله میمونی",
    "جشنواره فیلم فجر",
    "رونمایی از گوشی جدید اپل",
    "روش پخت غذا",
]

In [42]:
queries_class_prediction = []

for query in QUERIES:
    preprocessed_query = ' '.join(preprocessor.preprocess(query))
    prediction = pipe(preprocessed_query)[0]
    predicted_class_code = CATEGORIES_CLASSES[prediction['label']]
    probability = prediction['score']
    queries_class_prediction.append({
        'Query': query,
        'Predicted Category': CLASSES_CATEGORIES[predicted_class_code],
        'Probability': probability
    })

display(pd.DataFrame(queries_class_prediction))

,Query,Predicted Category,Probability
0,نتیجه توافق برجام,Politics,0.994476
1,حمله ارتش روسیه به اوکراین,World,0.985831
2,افزایش نرخ تورم در کشور,Economy,0.852276
3,آمار فوتی‌های کرونا,Society,0.993497
4,اقدامات شهرداری تهران,City,0.988388
5,صعود ایران به جام جهانی,Sport,0.987941
6,بیماری آبله میمونی,Science,0.957387
7,جشنواره فیلم فجر,Culture,0.989262
8,رونمایی از گوشی جدید اپل,IT,0.832700
9,روش پخت غذا,LifeSkills,0.987767


In [36]:
golden_labels = test_data['label'].to_numpy()

In [38]:
model_test_predictions = pd.DataFrame(pipe(test_data['text'].to_list())).label.apply(lambda x: CATEGORIES_CLASSES[x]).to_numpy()

In [120]:
mean_acc = np.average(model_test_predictions == golden_labels)
print(f"Mean Accurracy: {round(mean_acc, 3)}\n")

f1_macro = f1_score(golden_labels, model_test_predictions, labels=np.arange(10), average='macro')
print(f"F1-Macro: {round(f1_macro, 3)}\n")

confusion_mat = confusion_matrix(golden_labels, model_test_predictions, labels=np.arange(10))
confusion_mat_df = pd.DataFrame(confusion_mat).rename(columns=CLASSES_CATEGORIES, index=CLASSES_CATEGORIES)
display(confusion_mat_df)

Mean Accurracy: 0.844

F1-Macro: 0.8



,Politics,World,Economy,Society,City,Sport,Science,Culture,IT,LifeSkills
Politics,1990,78,77,105,23,9,4,34,2,4
World,108,268,5,12,0,9,30,5,2,13
Economy,75,9,1160,65,15,3,7,1,3,9
Society,84,6,52,1722,84,9,26,44,0,37
City,27,4,13,76,451,1,5,12,0,4
Sport,13,1,2,9,2,1163,1,16,1,4
Science,6,12,7,32,5,0,372,8,17,34
Culture,36,8,6,28,8,10,2,862,0,11
IT,1,2,2,1,0,0,13,4,40,2
LifeSkills,8,8,3,32,5,11,19,16,1,629


In [91]:
pipe('حمله‌ی روسیه به اوکراین')

[{'label': 'World', 'score': 0.9983450174331665}]

In [29]:
pipe('حمله‌ی روسیه به اوکراین')

[[{'label': 'Politics', 'score': 0.0029673657845705748},
  {'label': 'World', 'score': 0.9889428615570068},
  {'label': 'Economy', 'score': 0.0010848098900169134},
  {'label': 'Society', 'score': 7.766261842334643e-05},
  {'label': 'City', 'score': 0.00021785734861623496},
  {'label': 'Sport', 'score': 0.0014601512812078},
  {'label': 'Science', 'score': 0.0015758021036162972},
  {'label': 'Culture', 'score': 0.0011481213150545955},
  {'label': 'IT', 'score': 0.0013944440288469195},
  {'label': 'LifeSkills', 'score': 0.0011309089604765177}]]

In [92]:
pipe('صعود ایران به جام جهانی')

[{'label': 'Sport', 'score': 0.9994328618049622}]

In [30]:
pipe('صعود ایران به جام جهانی')

[[{'label': 'Politics', 'score': 0.0005841043894179165},
  {'label': 'World', 'score': 0.0008120812126435339},
  {'label': 'Economy', 'score': 0.005739080253988504},
  {'label': 'Society', 'score': 0.0006637150654569268},
  {'label': 'City', 'score': 0.0003584826772566885},
  {'label': 'Sport', 'score': 0.9879292249679565},
  {'label': 'Science', 'score': 0.0002101670834235847},
  {'label': 'Culture', 'score': 0.0003720947715919465},
  {'label': 'IT', 'score': 0.0004837738233618438},
  {'label': 'LifeSkills', 'score': 0.0028472174890339375}]]

In [96]:
pipe('چی بخورم تا کمی لاغر شم')

[{'label': 'LifeSkills', 'score': 0.999549925327301}]

In [31]:
pipe('چی بخورم تا کمی لاغر شم')

[[{'label': 'Politics', 'score': 0.00018790416652336717},
  {'label': 'World', 'score': 0.0014136965619400144},
  {'label': 'Economy', 'score': 0.000894360535312444},
  {'label': 'Society', 'score': 0.0008207164355553687},
  {'label': 'City', 'score': 0.000644539250060916},
  {'label': 'Sport', 'score': 0.001473690033890307},
  {'label': 'Science', 'score': 0.01829366572201252},
  {'label': 'Culture', 'score': 0.0006022172747179866},
  {'label': 'IT', 'score': 0.0010841957991942763},
  {'label': 'LifeSkills', 'score': 0.9745849967002869}]]

In [97]:
pipe('سرقت گوشی از کیف یک مرد چاق')

[{'label': 'Society', 'score': 0.9996367692947388}]

In [32]:
pipe('سرقت گوشی از کیف یک مرد چاق')

[[{'label': 'Politics', 'score': 0.0016436390578746796},
  {'label': 'World', 'score': 0.0017621704610064626},
  {'label': 'Economy', 'score': 0.0017307662637904286},
  {'label': 'Society', 'score': 0.9889982342720032},
  {'label': 'City', 'score': 0.0002893876808229834},
  {'label': 'Sport', 'score': 0.00022175326012074947},
  {'label': 'Science', 'score': 0.001095517072826624},
  {'label': 'Culture', 'score': 0.0007873851573094726},
  {'label': 'IT', 'score': 0.0001515335461590439},
  {'label': 'LifeSkills', 'score': 0.003319623414427042}]]

In [99]:
pipe('سرقت گوشی آیفون با نمایشگر اولد از کیف یک مرد چاق')

[{'label': 'Society', 'score': 0.9228408932685852}]

In [33]:
pipe('سرقت گوشی آیفون با نمایشگر اولد از کیف یک مرد چاق')

[[{'label': 'Politics', 'score': 0.009319602511823177},
  {'label': 'World', 'score': 0.03730259835720062},
  {'label': 'Economy', 'score': 0.03180873021483421},
  {'label': 'Society', 'score': 0.7370122075080872},
  {'label': 'City', 'score': 0.0014679653104394674},
  {'label': 'Sport', 'score': 0.0033028435427695513},
  {'label': 'Science', 'score': 0.10531005263328552},
  {'label': 'Culture', 'score': 0.004941267892718315},
  {'label': 'IT', 'score': 0.025610100477933884},
  {'label': 'LifeSkills', 'score': 0.04392459988594055}]]

In [34]:
pipe('سرقت گوشی آیفون با نمایشگر اولد از کیف پوتین')

[[{'label': 'Politics', 'score': 0.07910528033971786},
  {'label': 'World', 'score': 0.5980319380760193},
  {'label': 'Economy', 'score': 0.042239122092723846},
  {'label': 'Society', 'score': 0.02066550776362419},
  {'label': 'City', 'score': 0.0009063664474524558},
  {'label': 'Sport', 'score': 0.005482566077262163},
  {'label': 'Science', 'score': 0.06611823290586472},
  {'label': 'Culture', 'score': 0.007100971415638924},
  {'label': 'IT', 'score': 0.13817036151885986},
  {'label': 'LifeSkills', 'score': 0.042179565876722336}]]

In [22]:
pipe('مراسم رونمایی از گوشی جدید اپل')

[[{'label': 'Politics', 'score': 0.00015180022455751896},
  {'label': 'World', 'score': 0.0011100467527285218},
  {'label': 'Economy', 'score': 0.0003383722505532205},
  {'label': 'Society', 'score': 1.3191202015150338e-05},
  {'label': 'City', 'score': 0.00023218822025228292},
  {'label': 'Sport', 'score': 0.0004773465625476092},
  {'label': 'Science', 'score': 0.06702743470668793},
  {'label': 'Culture', 'score': 0.0008597681298851967},
  {'label': 'IT', 'score': 0.9295721650123596},
  {'label': 'LifeSkills', 'score': 0.00021759438095614314}]]

In [35]:
pipe('مراسم رونمایی از گوشی جدید اپل')

[[{'label': 'Politics', 'score': 0.0017105998704209924},
  {'label': 'World', 'score': 0.0036698956973850727},
  {'label': 'Economy', 'score': 0.0160268172621727},
  {'label': 'Society', 'score': 0.00024359303642995656},
  {'label': 'City', 'score': 0.0006743140402249992},
  {'label': 'Sport', 'score': 0.0010696231620386243},
  {'label': 'Science', 'score': 0.12873099744319916},
  {'label': 'Culture', 'score': 0.0025361115112900734},
  {'label': 'IT', 'score': 0.8410767912864685},
  {'label': 'LifeSkills', 'score': 0.004261353053152561}]]

In [100]:
pipe('سرقت گوشی پوتین توسط ترامپ')

[{'label': 'Politics', 'score': 0.886298418045044}]

In [36]:
pipe('سرقت گوشی پوتین توسط ترامپ')

[[{'label': 'Politics', 'score': 0.3795817494392395},
  {'label': 'World', 'score': 0.5742902159690857},
  {'label': 'Economy', 'score': 0.00623669195920229},
  {'label': 'Society', 'score': 0.0012188126565888524},
  {'label': 'City', 'score': 0.000506470852997154},
  {'label': 'Sport', 'score': 0.0015986207872629166},
  {'label': 'Science', 'score': 0.005829408299177885},
  {'label': 'Culture', 'score': 0.002252679318189621},
  {'label': 'IT', 'score': 0.019064098596572876},
  {'label': 'LifeSkills', 'score': 0.009421206079423428}]]

In [101]:
pipe('سرقت گوشی پوتین توسط رضا گلزار')

[{'label': 'Culture', 'score': 0.866288959980011}]

In [37]:
pipe('سرقت گوشی پوتین توسط رضا گلزار')

[[{'label': 'Politics', 'score': 0.009315567091107368},
  {'label': 'World', 'score': 0.004395921714603901},
  {'label': 'Economy', 'score': 0.0038018813356757164},
  {'label': 'Society', 'score': 0.3888435661792755},
  {'label': 'City', 'score': 0.0015483637107536197},
  {'label': 'Sport', 'score': 0.0035090455785393715},
  {'label': 'Science', 'score': 0.003538967575877905},
  {'label': 'Culture', 'score': 0.5069535970687866},
  {'label': 'IT', 'score': 0.0021129280794411898},
  {'label': 'LifeSkills', 'score': 0.07598016411066055}]]

In [102]:
pipe('سرقت پوتین')

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[{'label': 'World', 'score': 0.9596025943756104}]

In [109]:
pipe('سفر وزیر امور خارجه ایران به گلزار محلات')

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[{'label': 'Politics', 'score': 0.9959600567817688}]

In [44]:
pipe('سفر وزیر امور خارجه ایران به گلزار محلات')

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[[{'label': 'Politics', 'score': 0.691024899482727},
  {'label': 'World', 'score': 0.002618016442283988},
  {'label': 'Economy', 'score': 0.006856570485979319},
  {'label': 'Society', 'score': 0.212351992726326},
  {'label': 'City', 'score': 0.009226523339748383},
  {'label': 'Sport', 'score': 0.007143766153603792},
  {'label': 'Science', 'score': 0.0006506700301542878},
  {'label': 'Culture', 'score': 0.04556664451956749},
  {'label': 'IT', 'score': 0.000725195393897593},
  {'label': 'LifeSkills', 'score': 0.023835845291614532}]]

In [112]:
pipe('دعوای اردوغان و گلزار')

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[{'label': 'Politics', 'score': 0.719677209854126}]

In [38]:
pipe('دعوای اردوغان و گلزار')

[[{'label': 'Politics', 'score': 0.22526055574417114},
  {'label': 'World', 'score': 0.009653659537434578},
  {'label': 'Economy', 'score': 0.00161020178347826},
  {'label': 'Society', 'score': 0.034200165420770645},
  {'label': 'City', 'score': 0.004634971264749765},
  {'label': 'Sport', 'score': 0.0019638845697045326},
  {'label': 'Science', 'score': 0.00046728234156034887},
  {'label': 'Culture', 'score': 0.7130544781684875},
  {'label': 'IT', 'score': 0.0005071430350653827},
  {'label': 'LifeSkills', 'score': 0.00864777434617281}]]

In [113]:
pipe('دعوای اردوغان و رضا گلزار')

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[{'label': 'Culture', 'score': 0.9993233680725098}]

In [39]:
pipe('گلزار شهدا')

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[[{'label': 'Politics', 'score': 0.0012449071509763598},
  {'label': 'World', 'score': 0.0003142862697131932},
  {'label': 'Economy', 'score': 0.0010023785289376974},
  {'label': 'Society', 'score': 0.7528059482574463},
  {'label': 'City', 'score': 0.21576933562755585},
  {'label': 'Sport', 'score': 0.0018549913074821234},
  {'label': 'Science', 'score': 0.001124151749536395},
  {'label': 'Culture', 'score': 0.020494913682341576},
  {'label': 'IT', 'score': 0.0005638091242872179},
  {'label': 'LifeSkills', 'score': 0.00482518132776022}]]